In [1]:
# backtest_multi_stock.py (extended with toggleable indicators)
import pandas as pd
import numpy as np
import yfinance as yf
from custom_indicators import calculate_ema, calculate_supertrend

# -------------------- CONFIG --------------------
TICKER_LIST = [
    'ADANIENT.NS','ADANIPORTS.NS','APOLLOHOSP.NS','ASIANPAINT.NS','AXISBANK.NS',
    'BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BEL.NS','BHARTIARTL.NS',
    'CIPLA.NS','COALINDIA.NS','DRREDDY.NS','EICHERMOT.NS','ETERNAL.NS',
    'GRASIM.NS','HCLTECH.NS','HDFCBANK.NS','HDFCLIFE.NS','HEROMOTOCO.NS',
    'HINDALCO.NS','HINDUNILVR.NS','ICICIBANK.NS','ITC.NS','INDUSINDBK.NS',
    'INFY.NS','JSWSTEEL.NS','JIOFIN.NS','KOTAKBANK.NS','LT.NS','M&M.NS',
    'MARUTI.NS','NTPC.NS','NESTLEIND.NS','ONGC.NS','POWERGRID.NS','RELIANCE.NS',
    'SBILIFE.NS','SHRIRAMFIN.NS','SBIN.NS','SUNPHARMA.NS','TCS.NS','TATACONSUM.NS',
    'TATAMOTORS.NS','TATASTEEL.NS','TECHM.NS','TITAN.NS','TRENT.NS','ULTRACEMCO.NS','WIPRO.NS'
]

START_DATE = "2025-01-01"
END_DATE = "2025-08-31"
# Supertrend & EMA (existing)
ST_LENGTH = 10
ST_MULTIPLIER = 3.0
EMA_FAST_LENGTH = 9
EMA_SLOW_LENGTH = 15

# Backtest exit params
DEFAULT_TRAILING_STOP_PCT = 10.0
DEFAULT_HARD_STOP_PCT = 5.0
DEFAULT_MAX_HOLD_DAYS = 10

# Indicator toggles & params (change these to test combos)
INDICATORS = {
    "rsi": {
        "enabled": False,
        "length": 14,
        "buy_below": 40,    # require RSI below this for a buy confirmation (oversold)
        "sell_above": 70    # optional exit if RSI > this (overbought)
    },
    "macd": {
        "enabled": False,
        "fast": 12,
        "slow": 26,
        "signal": 9,
        # buy when macd > signal (bullish cross). 'momentum' param requires MACD - signal > threshold
        "momentum_threshold": 0.0
    },
    "adx": {
        "enabled": False,
        "length": 14,
        "min_adx": 20.0,   # require ADX above this to confirm trend strength
    },
    "sma": {
        "enabled": True,
        "length": 30,      # simple moving average - require price above SMA for buys
    },
    "bb": {
        "enabled": False,
        "length": 20,
        "stddev": 2.0,
        # buy when close crosses above lower band (optional)
    }
}

# How to combine indicator confirmations:
# 'all' => require all enabled indicators to confirm buy
# 'any' => require any one of the enabled indicators to confirm buy (in addition to supertrend+ema)
COMBINATION_MODE = 'any'  # 'all' or 'any'

# -------------------------------------------------


# -------------------- INDICATOR HELPERS --------------------
def calculate_rsi(df, length=14, column='Close'):
    """Compute RSI and place in df['RSI']"""
    delta = df[column].diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, adjust=False).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    rsi = 100 - (100 / (1 + rs))
    df['RSI'] = rsi.fillna(50)  # neutral default
    return df

def calculate_macd(df, fast=12, slow=26, signal=9, column='Close'):
    """Compute MACD line and signal line"""
    ema_fast = df[column].ewm(span=fast, adjust=False).mean()
    ema_slow = df[column].ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    df['MACD'] = macd_line
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

def calculate_sma(df, length=50, column='Close'):
    df[f'SMA_{length}'] = df[column].rolling(length).mean()
    return df

def calculate_bbands(df, length=20, stddev=2, column='Close'):
    sma = df[column].rolling(length).mean()
    std = df[column].rolling(length).std()
    df['BB_middle'] = sma
    df['BB_upper'] = sma + (stddev * std)
    df['BB_lower'] = sma - (stddev * std)
    return df

def calculate_adx(df, n=14):
    """
    Calculate +DI, -DI and ADX (classic Wilder's method).
    Places in df['ADX'], df['+DI'], df['-DI'].
    """
    high = df['High']
    low = df['Low']
    close = df['Close']

    # True Range
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # Directional Movement
    up_move = high.diff()
    down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)

    # Smooth TR, +DM, -DM (Wilder's smoothing)
    tr_smooth = pd.Series(tr).rolling(window=n).sum()
    plus_dm_smooth = pd.Series(plus_dm).rolling(window=n).sum()
    minus_dm_smooth = pd.Series(minus_dm).rolling(window=n).sum()

    # Avoid divide by zero
    plus_di = 100 * (plus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    minus_di = 100 * (minus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)).replace([np.inf, -np.inf], 0) * 100
    adx = dx.rolling(window=n).mean()

    df['+DI'] = plus_di
    df['-DI'] = minus_di
    df['ADX'] = adx
    return df
# ------------------------------------------------------------


# -------------------- DATA & PREP --------------------
def get_stock_data(ticker, start, end, indicators=INDICATORS):
    df = yf.download(ticker, start=start, interval="1d", end=end, auto_adjust=True, progress=False, threads=True, multi_level_index=False)
    if df.empty:
        print(f"No data found for {ticker}. Skipping.")
        return None

    # Existing EMAs & Supertrend (using your custom functions)
    df['EMA_fast'] = calculate_ema(df, EMA_FAST_LENGTH)
    df['EMA_slow'] = calculate_ema(df, EMA_SLOW_LENGTH)
    df = calculate_supertrend(df, ST_LENGTH, ST_MULTIPLIER)

    # Additional indicators based on toggles
    if indicators.get('rsi', {}).get('enabled', False):
        df = calculate_rsi(df, length=indicators['rsi'].get('length', 14))

    if indicators.get('macd', {}).get('enabled', False):
        df = calculate_macd(df,
                            fast=indicators['macd'].get('fast', 12),
                            slow=indicators['macd'].get('slow', 26),
                            signal=indicators['macd'].get('signal', 9))

    if indicators.get('adx', {}).get('enabled', False):
        df = calculate_adx(df, n=indicators['adx'].get('length', 14))

    if indicators.get('sma', {}).get('enabled', False):
        df = calculate_sma(df, length=indicators['sma'].get('length', 50))

    if indicators.get('bb', {}).get('enabled', False):
        df = calculate_bbands(df, length=indicators['bb'].get('length', 20), stddev=indicators['bb'].get('stddev', 2.0))

    df.dropna(inplace=True)
    return df
# --------------------------------------------------


# -------------------- SIGNAL COMBINATION --------------------
def indicator_buy_checks(row, prev_row, indicators=INDICATORS):
    """
    Evaluate each enabled indicator's buy confirmation.
    Returns dict {indicator_name: True/False} (True means indicator confirms buy).
    """
    results = {}

    # RSI: buy if RSI <= buy_below (oversold)
    if indicators.get('rsi', {}).get('enabled', False):
        r = row.get('RSI', np.nan)
        results['rsi'] = (r <= indicators['rsi'].get('buy_below', 40))

    # MACD: buy when MACD > signal and optionally histogram > threshold
    if indicators.get('macd', {}).get('enabled', False):
        macd = row.get('MACD', 0)
        sig = row.get('MACD_signal', 0)
        hist = row.get('MACD_hist', 0)
        thr = indicators['macd'].get('momentum_threshold', 0.0)
        results['macd'] = (macd > sig) and (hist > thr)

    # ADX: buy if ADX >= min_adx AND +DI > -DI (trend strength + direction)
    if indicators.get('adx', {}).get('enabled', False):
        adx = row.get('ADX', 0)
        plus = row.get('+DI', 0)
        minus = row.get('-DI', 0)
        min_adx = indicators['adx'].get('min_adx', 20.0)
        results['adx'] = (adx >= min_adx) and (plus > minus)

    # SMA: buy if price above SMA
    if indicators.get('sma', {}).get('enabled', False):
        sma_col = f"SMA_{indicators['sma'].get('length', 50)}"
        results['sma'] = (row['Close'] > row.get(sma_col, np.nan))

    # BB: buy if close crosses above lower band (simple)
    if indicators.get('bb', {}).get('enabled', False):
        results['bb'] = (prev_row is not None) and (prev_row['Close'] < prev_row.get('BB_lower', np.nan)) and (row['Close'] > row.get('BB_lower', np.nan))

    return results

def combine_indicator_signals(ind_results, mode='all'):
    enabled = [k for k in ind_results.keys()]
    if not enabled:
        return True  # nothing to check
    truths = [v for v in ind_results.values()]
    if mode == 'all':
        return all(truths)
    else:
        return any(truths)
# --------------------------------------------------


# -------------------- BACKTEST --------------------
def run_backtest(df, ticker,
                 trailing_stop_loss_pct=DEFAULT_TRAILING_STOP_PCT,
                 hard_stop_loss_pct=DEFAULT_HARD_STOP_PCT,
                 max_holding_days=DEFAULT_MAX_HOLD_DAYS,
                 indicators=INDICATORS,
                 combination_mode=COMBINATION_MODE):
    in_position = False
    trades = []
    entry_price = 0
    entry_date = None

    peak_price = 0
    trailing_stop_price = 0
    hard_stop_price = 0
    days_in_trade = 0

    tsl_multiplier = 1 - (trailing_stop_loss_pct / 100)
    hsl_multiplier = 1 - (hard_stop_loss_pct / 100)

    print(f"--- Running backtest for {ticker} (TSL: {trailing_stop_loss_pct}%, HSL: {hard_stop_loss_pct}%, Max Days: {max_holding_days}) ---")

    # iterate rows by index, keep prev row for cross checks (BB / previous values)
    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]
        current_row = df.iloc[i]

        # base supertrend + ema condition (your existing)
        is_bullish_state = current_row['supertrend_direction'] == 1 and current_row['EMA_fast'] > current_row['EMA_slow']
        was_bullish_state = prev_row['supertrend_direction'] == 1 and prev_row['EMA_fast'] > prev_row['EMA_slow']
        buy_signal_base = is_bullish_state and not was_bullish_state
        stop_loss_signal = prev_row['supertrend_direction'] == 1 and current_row['supertrend_direction'] == -1

        # Evaluate indicator confirmations
        ind_results = indicator_buy_checks(current_row, prev_row, indicators=indicators)
        ind_combined = combine_indicator_signals(ind_results, mode=combination_mode)

        # Final buy signal: base signal AND indicator combination (if any indicator enabled)
        enabled_any = any(v.get('enabled', False) for v in indicators.values())
        buy_signal = buy_signal_base and (ind_combined if enabled_any else True)

        if not in_position and buy_signal:
            entry_price = current_row['Close']
            in_position = True
            entry_date = current_row.name

            peak_price = entry_price
            trailing_stop_price = peak_price * tsl_multiplier
            hard_stop_price = entry_price * hsl_multiplier
            days_in_trade = 0
            # log
            print(f"  Enter: {entry_date.strftime('%Y-%m-%d %H:%M')} @ {entry_price:.2f} | indicators: {ind_results}")

        elif in_position:
            days_in_trade += 1

            # Update peak price and trailing stop (on intrabar high)
            if current_row['High'] > peak_price:
                peak_price = current_row['High']
                trailing_stop_price = peak_price * tsl_multiplier

            exit_price = 0
            exit_reason = None

            # 1. Hard stop
            if current_row['Low'] <= hard_stop_price:
                exit_price = hard_stop_price
                exit_reason = "HSL"
            # 2. Trailing stop
            elif current_row['Low'] <= trailing_stop_price:
                exit_price = trailing_stop_price
                exit_reason = "TSL"
            # 3. Time-based stop
            elif days_in_trade >= max_holding_days:
                exit_price = current_row['Close']
                exit_reason = "Time"
            # 4. Supertrend cross (your existing)
            elif stop_loss_signal:
                exit_price = current_row['Close']
                exit_reason = "SL"
            # 5. Indicator-based exit examples:
            else:
                # RSI overbought exit
                if indicators.get('rsi', {}).get('enabled', False):
                    r = current_row.get('RSI', np.nan)
                    if r >= indicators['rsi'].get('sell_above', 70):
                        exit_price = current_row['Close']
                        exit_reason = "RSI_exit"

                # ADX weakening (optional): ADX falls below threshold -> exit
                if exit_price == 0 and indicators.get('adx', {}).get('enabled', False):
                    adx = current_row.get('ADX', np.nan)
                    min_adx = indicators['adx'].get('min_adx', 20.0)
                    if adx < min_adx:
                        exit_price = current_row['Close']
                        exit_reason = "ADX_weak"

                # MACD bearish cross
                if exit_price == 0 and indicators.get('macd', {}).get('enabled', False):
                    # if MACD crosses below signal on this bar (prev macd > prev signal and now macd < signal)
                    prev_macd = prev_row.get('MACD', 0)
                    prev_sig = prev_row.get('MACD_signal', 0)
                    cur_macd = current_row.get('MACD', 0)
                    cur_sig = current_row.get('MACD_signal', 0)
                    if (prev_macd > prev_sig) and (cur_macd < cur_sig):
                        exit_price = current_row['Close']
                        exit_reason = "MACD_cross_down"

            if exit_price > 0:
                in_position = False
                exit_date = current_row.name
                percent_return = ((exit_price - entry_price) / entry_price) * 100

                trades.append({
                    "ticker": ticker,
                    "entry_date": entry_date, "exit_date": exit_date,
                    "entry_price": entry_price, "exit_price": exit_price,
                    "return_%": percent_return, "exit_reason": exit_reason,
                    "days_held": days_in_trade
                })
                print(f"  Trade Closed ({exit_reason}): {exit_date.strftime('%Y-%m-%d %H:%M')} | Return: {percent_return:.2f}% | Days: {days_in_trade}")

    return trades
# --------------------------------------------------


# -------------------- SUMMARY --------------------
def print_summary(all_trades):
    if not all_trades:
        print("\nNo trades were executed during the backtest period.")
        return

    trades_df = pd.DataFrame(all_trades)
    wins = trades_df[trades_df['return_%'] > 0]
    total_trades = len(trades_df)
    win_rate = (len(wins) / total_trades) * 100 if total_trades > 0 else 0

    hsl_exits = len(trades_df[trades_df['exit_reason'] == 'HSL'])
    tsl_exits = len(trades_df[trades_df['exit_reason'] == 'TSL'])
    time_exits = len(trades_df[trades_df['exit_reason'] == 'Time'])
    sl_exits = len(trades_df[trades_df['exit_reason'] == 'SL'])
    other_exits = total_trades - (hsl_exits + tsl_exits + time_exits + sl_exits)

    print(f"\nSummary:")
    print(f"  Total Trades: {total_trades}")
    print(f"  Win Rate: {win_rate:.2f}%")
    print(f"  Avg. Holding Period: {trades_df['days_held'].mean():.1f} bars")
    print("\n  Exit Reasons:")
    print(f"    Hard Stop-Loss (HSL): {hsl_exits}")
    print(f"    Trailing Stop-Loss (TSL): {tsl_exits}")
    print(f"    Time-Based Stop (Time): {time_exits}")
    print(f"    Supertrend Stop (SL): {sl_exits}")
    print(f"    Other Exits (RSI/MACD/ADX...): {other_exits}")
    print("-" * 30)
# --------------------------------------------------


# -------------------- MAIN --------------------
if __name__ == "__main__":
    overall_trades = []
    for stock_ticker in TICKER_LIST:
        stock_df = get_stock_data(stock_ticker, START_DATE, END_DATE, indicators=INDICATORS)
        if stock_df is not None:
            executed_trades = run_backtest(
                stock_df,
                stock_ticker,
                trailing_stop_loss_pct=DEFAULT_TRAILING_STOP_PCT,
                hard_stop_loss_pct=DEFAULT_HARD_STOP_PCT,
                max_holding_days=DEFAULT_MAX_HOLD_DAYS,
                indicators=INDICATORS,
                combination_mode=COMBINATION_MODE
            )
            overall_trades.extend(executed_trades)

    print("\n\n" + "="*50)
    print("Overall Portfolio Backtest Summary")
    print("="*50)
    print_summary(overall_trades)

    if overall_trades:
        results_df = pd.DataFrame(overall_trades)
        results_df.to_csv('backtest_results_advanced.csv', index=False)
        print("\n✅ Backtest results saved to backtest_results_advanced.csv")
    else:
        print("\nNo trades to save to CSV.")


--- Running backtest for ADANIENT.NS (TSL: 10.0%, HSL: 5.0%, Max Days: 10) ---
  Enter: 2025-03-19 00:00 @ 2317.46 | indicators: {'sma': np.True_}
  Trade Closed (Time): 2025-04-03 00:00 | Return: 3.97% | Days: 10
  Enter: 2025-04-11 00:00 @ 2320.21 | indicators: {'sma': np.True_}
  Trade Closed (Time): 2025-04-29 00:00 | Return: 0.40% | Days: 10
  Enter: 2025-05-05 00:00 @ 2454.25 | indicators: {'sma': np.True_}
  Trade Closed (HSL): 2025-05-07 00:00 | Return: -5.00% | Days: 2
  Enter: 2025-05-13 00:00 @ 2439.95 | indicators: {'sma': np.True_}
  Trade Closed (Time): 2025-05-27 00:00 | Return: 4.06% | Days: 10
  Enter: 2025-06-27 00:00 @ 2646.30 | indicators: {'sma': np.True_}
  Trade Closed (Time): 2025-07-11 00:00 | Return: -3.31% | Days: 10
--- Running backtest for ADANIPORTS.NS (TSL: 10.0%, HSL: 5.0%, Max Days: 10) ---
  Enter: 2025-03-19 00:00 @ 1167.82 | indicators: {'sma': np.True_}
  Trade Closed (Time): 2025-04-03 00:00 | Return: 2.25% | Days: 10
  Enter: 2025-04-15 00:00 @ 12